In [1]:
import logging
from argparse import ArgumentParser
from functools import partial
from pathlib import Path
from transformers  import AutoTokenizer
import datasets as ds
import numpy as np
from rich.progress import track

log = logging.getLogger(__name__)

/Users/mac/Documents/LLMs/llm.pth/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def filter(example):
    return example["n_labels"] > 0

In [3]:
def preprocess(example, tokenizer, max_seq_len: int):
    input_ids = [tokenizer.eos_token_id]
    label_mask = [False]

    for msg in example["messages"]:
        role_tokens = tokenizer.encode(f"<|{msg['role']}|>\n", add_special_tokens=False)
        label_mask += [False] * len(role_tokens)
        input_ids += role_tokens

        if msg["role"] == "assistant":
            content_tokens = tokenizer.encode(
                msg["content"].strip() + tokenizer.eos_token + "\n", add_special_tokens=False
            )
            label_mask += [True] * len(content_tokens)
            # mask out the last '\n'
            assert content_tokens[-2] == tokenizer.eos_token_id
            label_mask[-1] = False
        else:
            content_tokens = tokenizer.encode(msg["content"].strip() + "\n", add_special_tokens=False)
            label_mask += [False] * len(content_tokens)
        input_ids += content_tokens

    input_ids = input_ids[:max_seq_len]
    label_mask = label_mask[:max_seq_len]

    if len(input_ids) < max_seq_len:
        pad_len = max_seq_len - len(input_ids)
        input_ids += [tokenizer.pad_token_id] * pad_len
        label_mask += [False] * pad_len

    assert len(input_ids) == len(label_mask)
    n_labels = sum(label_mask)

    return {"input_ids": input_ids, "label_mask": label_mask, "n_labels": n_labels}

In [4]:
s = 2048
eos = 50279
pad = 1
num_proc=8
seq_len=2048
output_dir = "./out"

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
dataset = ds.load_dataset("allenai/tulu-v2-sft-mixture", split="train")

log.info("Tokenizing dataset...")
dataset = dataset.map(
    partial(preprocess, tokenizer=tokenizer, max_seq_len=seq_len),
    batched=False,
    remove_columns=["dataset", "id", "messages"],
    num_proc=num_proc,  # type: ignore
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map (num_proc=8): 100%|██████████| 326154/326154 [03:20<00:00, 1628.21 examples/s]


In [6]:
log.info("Filtering dataset...")
n = len(dataset)  # type: ignore
dataset = dataset.filter(filter, batched=False, num_proc=num_proc)  # type: ignore
log.info(f"Filtered out {n - len(dataset):,d} examples")

log.info("Counting tokens...")
total_tokens = 0
for ex in track(dataset):
    assert len(ex["input_ids"]) == seq_len  # type: ignore
    total_tokens += len(ex["input_ids"])  # type: ignore
log.info(f"Total tokens: {total_tokens:,d}")


Filter (num_proc=8): 100%|██████████| 326154/326154 [00:46<00:00, 7030.42 examples/s] 


/Users/mac/Documents/LLMs/llm.pth/.venv/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [7]:
log.info(f"Saving results to '{output_dir}'...")
output_dir = Path(output_dir)
output_dir.mkdir(exist_ok=True, parents=True)

input_ids_file = np.memmap(
    str(output_dir / "input_ids.npy"), dtype=np.uint16, mode="w+", shape=(total_tokens,)
)
label_mask_file = np.memmap(
    str(output_dir / "label_mask.npy"), dtype=np.bool_, mode="w+", shape=(total_tokens,)
)

In [26]:
offset = 10
for ex in track(dataset):
    ex_len = len(ex["input_ids"])  # type: ignore
    print(ex["input_ids"])
    input_ids_file[offset : offset + 200] = ex["input_ids"][0:200]  # type: ignore
    label_mask_file[offset : offset + ex_len] = ex["label_mask"]  # type: ignore
    offset += ex_len
input_ids_file.flush()
label_mask_file.flush()

log.info("Done!")

[50256, 27, 91, 7220, 91, 29, 198, 24361, 25, 402, 6814, 129, 226, 8135, 357, 11, 2162, 2679, 25, 13538, 837, 1267,
318, 257, 15945, 1748, 319, 262, 32882, 7051, 13, 632, 318, 262, 3139, 286, 262, 350, 12057, 38336, 20687, 452, 
4147, 1056, 11, 12873, 338, 10033, 384, 499, 419, 290, 318, 635, 262, 7372, 286, 262, 1499, 338, 5544, 12, 28209, 
27953, 1989, 13, 220, 198, 198, 464, 1748, 7363, 319, 262, 8372, 5743, 286, 402, 6814, 129, 226, 8135, 4696, 357, 
1659, 262, 32882, 6896, 828, 287, 257, 369, 5945, 341, 351, 262, 1748, 286, 402, 67, 2047, 544, 11, 41900, 3240, 
286, 35643, 313, 11, 290, 20154, 5348, 11, 543, 1978, 1296, 257, 27953, 1989, 1444, 262, 833, 8467, 5855, 2898, 
10205, 73, 11632, 459, 78, 12340, 351, 257, 3265, 13885, 352, 13, 19, 1510, 13, 402, 6814, 129, 226, 8135, 2346, 
468, 257, 3265, 286, 34091, 11, 42363, 357, 20588, 2321, 828, 1642, 340, 262, 4387, 1748, 287, 262, 350, 12057, 
5411, 3814, 286, 8342, 12873, 13, 220, 198, 198, 38, 6814, 129, 226, 8135, 318, 262, 3139, 286, 402, 6814, 129, 
226, 8135, 350, 12057, 5411, 290, 262, 4387, 1748, 286, 20578, 549, 544, 13, 2080, 663, 15587, 355, 257, 15945, 
32753, 28415, 287, 262, 32614, 82, 416, 337, 444, 89, 7204, 314, 286, 12873, 11, 262, 1748, 338, 2106, 318, 3716, 
11, 351, 9574, 286, 15945, 3896, 11, 9574, 286, 1736, 31562, 393, 2679, 3896, 11, 290, 9574, 286, 21851, 393, 2116,
12, 25135, 355, 257, 366, 5787, 1748, 1911, 14307, 262, 995, 9976, 11, 262, 3232, 2254, 286, 6035, 38262, 373, 287,
257, 17112, 6441, 351, 12873, 290, 373, 5140, 1022, 2679, 3687, 1736, 31269, 290, 262, 523, 12, 7174, 15945, 48994,
13, 220, 198, 198, 38, 6814, 129, 226, 8135, 7363, 379, 262, 5422, 286, 262, 6543, 41615, 6909, 5866, 11, 5884, 
284, 262, 12592, 14246, 4914, 11, 257, 8478, 287, 262, 25979, 286, 262, 6716, 569, 396, 4712, 5866, 11, 543, 44368,
3126, 1411, 286, 12873, 290, 20417, 402, 6814, 129, 226, 8135, 351, 262, 15945, 3139, 11, 32955, 13, 17083, 351, 
262, 6716, 2493, 286, 402, 67, 2047, 544, 11, 402, 6814, 129, 226, 8135, 318, 635, 281, 1593, 7593, 3641, 13, 554, 
262, 2739, 6046, 21449, 340, 373, 281, 1593, 384, 499, 419, 290, 4074, 16894, 3240, 11, 290, 287, 262, 1478, 400, 
290, 1315, 400, 10675, 257, 2888, 286, 262, 9530, 325, 1512, 4041, 13, 352, 13, 9022, 15489, 318, 402, 67, 34738, 
319, 30, 198, 17, 13, 9022, 5417, 318, 326, 15489, 636, 286, 30, 198, 18, 13, 1148, 402, 67, 34738, 257, 3139, 30, 
198, 19, 13, 3226, 644, 30, 198, 20, 13, 5338, 373, 262, 717, 22740, 30, 198, 21, 13, 6530, 530, 286, 262, 2678, 
326, 468, 8879, 340, 30, 198, 22, 13, 843, 1194, 30, 198, 23, 13, 843, 530, 517, 30, 198, 24, 13, 7875, 340, 1683, 
587, 739, 2116, 12, 25135, 30, 198, 940, 13, 1867, 373, 340, 1444, 1022, 262, 995, 9976, 30, 198, 1157, 13, 1148, 
340, 1474, 257, 7850, 30, 198, 1065, 13, 9022, 530, 30, 198, 1485, 13, 9022, 318, 5884, 284, 5633, 44290, 257, 
25840, 1351, 286, 7429, 13, 198, 2466, 198, 32, 25840, 286, 7429, 25, 352, 13, 402, 6814, 129, 226, 8135, 4696, 
198, 17, 13, 262, 32882, 6896, 198, 18, 13, 3363, 198, 19, 13, 350, 12057, 38336, 20687, 452, 4147, 1056, 198, 20, 
13, 337, 444, 89, 7204, 314, 198, 21, 13, 12873, 198, 22, 13, 2679, 198, 23, 13, 1736, 31562, 198, 24, 13, 3363, 
198, 940, 13, 6035, 38262, 198, 1157, 13, 3363, 198, 1065, 13, 262, 6543, 41615, 6909, 5866, 198, 1485, 13, 262, 
12592, 14246, 4914, 198, 48, 25, 15031, 11, 11565, 357, 18474, 8, 1377, 1081, 262, 520, 13, 5593, 23200, 286, 
15031, 49566, 3217, 706, 12513, 286, 8536, 625, 262, 10800, 4395, 286, 281, 24270, 2042, 6036, 11, 262, 1808, 3793,
25, 6350, 338, 262, 1644, 3818, 508, 5954, 262, 7616, 30, 220, 198, 198, 12710, 263, 26203, 8127, 11, 2579, 11, 
2823, 3899, 4373, 319, 2932, 860, 13, 383, 4395, 16752, 1528, 286, 6590, 8536, 287, 15031, 355, 5085, 12284, 465, 
3251, 13, 220, 198, 198, 19117, 30748, 2717, 290, 1957, 10986, 389, 739, 835, 11, 290, 8127, 1377, 508, 468, 2722, 
1918, 7432, 1377, 468, 12120, 422, 1171, 1570, 13, 220, 198, 198, 29168, 273, 6266, 3197, 2902, 286, 2351, 4

[50256, 27, 91, 7220, 91, 29, 198, 48, 25, 317, 13, 50, 13, 29600, 11, 8900, 11, 3700, 39651, 12375, 26, 3700, 
39651, 12375, 11, 4082, 27271, 11, 6182, 198, 32, 25, 3700, 39651, 12375, 11, 508, 373, 4642, 287, 6182, 11, 318, 
262, 8900, 286, 317, 13, 50, 13, 29600, 13, 198, 48, 25, 14562, 21146, 32115, 11, 968, 8221, 11, 318, 7841, 5189, 
11, 10596, 3418, 11, 968, 8221, 198, 32, 25, 14562, 21146, 32115, 318, 257, 42823, 287, 10596, 3418, 11, 968, 8221,
11, 1578, 1829, 13, 198, 48, 25, 15079, 3876, 34004, 11, 2099, 11, 311, 13, 32, 13, 357, 10215, 1819, 341, 1776, 
15079, 3876, 34004, 11, 1271, 5189, 29733, 2841, 11, 838, 22800, 198, 32, 25, 15079, 3876, 34004, 318, 281, 311, 
13, 32, 13, 12017, 543, 24803, 838, 11, 22800, 661, 13, 198, 48, 25, 12551, 11, 3303, 11, 3594, 3303, 26, 13709, 
321, 3112, 25907, 11, 3814, 11, 12551, 198, 32, 25, 198, 27, 91, 562, 10167, 91, 29, 198, 15823, 318, 530, 286, 
262, 8950, 9635, 287, 12551, 810, 38762, 321, 3112, 25907, 2058, 422, 13, 50256, 198, None, None, None, None, None,
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [20]:
list = [1,2,3,4]
x=2
y=1
print(list[y:x])
list2=[1,4,4]
list[0:1] = list2
print(list)

[2]
[1, 4, 4, 2, 3, 4]
